In [1]:
pip install dash plotly pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import os

D:\DHA SUFFA UNIVERSITY\C THIRD SEMSETER\downloads software for labs\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout 
app.layout = html.Div([
    dcc.Graph(id='live-update-graph'),
    dcc.Dropdown(
        id='attribute-dropdown',
        
        value='TTC9B'  # Default value
    ),
    dcc.Interval(
        id='interval-component',
        interval=5*1000,  # in milliseconds
        n_intervals=0
    )
])

In [4]:

# Function to load data
def load_data():
    file_path = r'C:\Users\Tariq\Documents\health_care_chronos.csv'
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file at {file_path} does not exist.")
    df = pd.read_csv(file_path)
    return df

In [5]:
# Populate the dropdown options 
@app.callback(
    Output('attribute-dropdown', 'options'),
    [Input('interval-component', 'n_intervals')]
)
def update_dropdown_options(n_intervals):
    df = load_data()
    # Generate dropdown options based on dataset columns
    options = [{'label': col, 'value': col} for col in df.columns if col != 'Sample_ID']
    return options

In [6]:
# Define the callback function to update the graph
@app.callback(
    Output('live-update-graph', 'figure'),
    [Input('interval-component', 'n_intervals'),
     Input('attribute-dropdown', 'value')],
    prevent_initial_call='initial_duplicate'  
)
def update_graph(n_intervals, selected_attribute):
    df = load_data()
    
    # Check if selected_attribute is valid
    if selected_attribute not in df.columns:
        raise ValueError(f"The selected attribute '{selected_attribute}' is not in the dataset columns.")
    
    # Create the line plot
    fig = px.line(df, x='Sample_ID', y=selected_attribute, title=f'{selected_attribute} Over Time')

    fig.update_traces(line=dict(color='Green'))  
    
    return fig


In [7]:
# Run the Dash app on a different port
if __name__ == '__main__':
    app.run_server(debug=True, port=8082)
